In [44]:
import sqlite3

def execute_req(sql_text, *args):
    conn = sqlite3.connect('tysql.sqlite')
    curs = conn.cursor()
    result = curs.execute(sql_text,(*args,)).fetchall()
    conn.close()
    return result

def execute_with_print(sql_text):
    r = execute_req(sql_text)
    for row in r:
        print(row)

In [45]:
# List all tables in db

all_tables = execute_req('SELECT name from sqlite_master where type= "table"')
all_tables = [t[0] for t in all_tables]
print('Таблиці у БД:')
for t in all_tables:
    print(t)

Таблиці у БД:
Customers
OrderItems
Orders
Products
Vendors


In [46]:
# List all columns and their properties

print('Інформація про поля таблиць:\n')
print('#, name, value_type, is_nullable, column_default, foreign key\n')
for t in all_tables:
    print(t)
    sql_text = "PRAGMA table_info (%s)" %t
    execute_with_print(sql_text)
    print()



Інформація про поля таблиць:

#, name, value_type, is_nullable, column_default, foreign key

Customers
(0, 'cust_id', 'char(10)', 1, None, 1)
(1, 'cust_name', 'char(50)', 1, None, 0)
(2, 'cust_address', 'char(50)', 0, None, 0)
(3, 'cust_city', 'char(50)', 0, None, 0)
(4, 'cust_state', 'char(5)', 0, None, 0)
(5, 'cust_zip', 'char(10)', 0, None, 0)
(6, 'cust_country', 'char(50)', 0, None, 0)
(7, 'cust_contact', 'char(50)', 0, None, 0)
(8, 'cust_email', 'char(255)', 0, None, 0)

OrderItems
(0, 'order_num', 'int', 1, None, 1)
(1, 'order_item', 'int', 1, None, 2)
(2, 'prod_id', 'char(10)', 1, None, 0)
(3, 'quantity', 'int', 1, None, 0)
(4, 'item_price', 'decimal(8,2)', 1, None, 0)

Orders
(0, 'order_num', 'int', 1, None, 1)
(1, 'order_date', 'datetime', 1, None, 0)
(2, 'cust_id', 'char(10)', 1, None, 0)

Products
(0, 'prod_id', 'char(10)', 1, None, 1)
(1, 'vend_id', 'char(10)', 1, None, 0)
(2, 'prod_name', 'char(255)', 1, None, 0)
(3, 'prod_price', 'decimal(8,2)', 1, None, 0)
(4, 'prod_desc

In [62]:
# Information about primary and foreign keys
'''
Даний запит поветрає CREATE statement, за яким було створено таблицю
tbl_name = назва таблиці
'''
sql_text="""SELECT sql FROM sqlite_master
    WHERE tbl_name = ? AND type = 'table'"""

for t in all_tables:
    '''
    #після create_statement = execute_req(sql_text, t,)[0][0] отримуємо список з 1 елементом, який є кортежем
    щоб дістатися до стрічки у кортежі звертаємось до елемента [0][0]
    '''
    create_statement = execute_req(sql_text, t,)[0][0]  
    
    
    '''
    Шукаємо у CREATE statement 'PRIMARY KEY'. Якщо у таблиці є FOREIGN KEY, це буде вказано після PRIMARY KEY 
    '''
    pk = create_statement[create_statement.find('PRIMARY KEY'):].rstrip(',\n)')+')' 
    print(t+'\n'+ pk+'\n')




Customers
PRIMARY KEY (cust_id)

OrderItems
PRIMARY KEY (order_num, order_item)                   ,
  FOREIGN KEY (order_num) REFERENCES Orders (order_num) ,
  FOREIGN KEY (prod_id) REFERENCES Products (prod_id)

Orders
PRIMARY KEY (order_num)      ,
  FOREIGN KEY (cust_id) REFERENCES Customers (cust_id)

Products
PRIMARY KEY (prod_id)             ,
  FOREIGN KEY (vend_id) REFERENCES Vendors (vend_id)

Vendors
PRIMARY KEY (vend_id)



In [68]:
# Список всіх cust_id з Customers:
sql_text_1 = """
SELECT cust_id
FROM Customers
"""
print('Customer ids from Customers\n')
execute_with_print(sql_text_1)

Customer ids from Customers

('1000000001',)
('1000000002',)
('1000000003',)
('1000000004',)
('1000000005',)


In [67]:
# Вся таблиця Customer Table:

sql_text_2 = """
SELECT *
FROM Customers
"""
print('All Customers table:\n')

execute_with_print(sql_text_2)


All Customers table:

('1000000001', 'Village Toys', '200 Maple Lane', 'Detroit', 'MI', '44444', 'USA', 'John Smith', 'sales@villagetoys.com')
('1000000002', 'Kids Place', '333 South Lake Drive', 'Columbus', 'OH', '43333', 'USA', 'Michelle Green', None)
('1000000003', 'Fun4All', '1 Sunny Place', 'Muncie', 'IN', '42222', 'USA', 'Jim Jones', 'jjones@fun4all.com')
('1000000004', 'Fun4All', '829 Riverside Drive', 'Phoenix', 'AZ', '88888', 'USA', 'Denise L. Stephens', 'dstephens@fun4all.com')
('1000000005', 'The Toy Store', '4545 53rd Street', 'Chicago', 'IL', '54545', 'USA', 'Kim Howard', None)


In [70]:
# Список клієнтів (cust_name) відсортованих від Z до A:

sql_text_3 = """
SELECT cust_name
FROM Customers
ORDER BY cust_name DESC
"""
print('Cust_name відсортовані від Z до A\n')
execute_with_print(sql_text_3)



Cust_name відсортовані від Z до A

('Village Toys',)
('The Toy Store',)
('Kids Place',)
('Fun4All',)
('Fun4All',)


In [71]:
# Таблиця клієнтів та замовлень (cust_id, order_num) відсортованих по клієнту та даті замовлення:

sql_text_4 = """
SELECT cust_id, order_num
FROM Orders
ORDER BY cust_id, order_date
"""
print('# Таблиця клієнтів та замовлень (cust_id, order_num) відсортованих по клієнту та даті замовлення:\n')
execute_with_print(sql_text_4)



# Таблиця клієнтів та замовлень (cust_id, order_num) відсортованих по клієнту та даті замовлення:

('1000000001', 20009)
('1000000001', 20005)
('1000000003', 20006)
('1000000004', 20007)
('1000000005', 20008)


In [76]:
# Таблиця на основі Items з кількістю та вартістю товару. Відсортуйте в порядку спадання по кількості товару

sql_text_5 = """
SELECT quantity, item_price
FROM OrderItems
ORDER BY quantity DESC
"""

print('Таблиця на основі Items з кількістю та вартістю товару. Відсортуйте в порядку спадання по кількості товару\n')
execute_with_print(sql_text_5)

Таблиця на основі Items з кількістю та вартістю товару. Відсортуйте в порядку спадання по кількості товару

(250, 2.49)
(250, 2.49)
(250, 2.49)
(100, 5.49)
(100, 10.99)
(100, 2.99)
(100, 2.99)
(100, 2.99)
(50, 11.49)
(50, 4.49)
(20, 5.99)
(10, 8.99)
(10, 11.99)
(10, 3.49)
(10, 3.49)
(10, 3.49)
(5, 4.99)
(5, 11.99)


In [78]:
# Товар з таблиці Products, ціна якого становить 9.49

sql_text_6 = """
SELECT prod_name, prod_price
FROM Products
WHERE prod_price = 9.49
"""
print('Товар з таблиці Products, ціна якого становить 9.49\n')
execute_with_print(sql_text_6)

Товар з таблиці Products, ціна якого становить 9.49

('King doll', 9.49)
('Queen doll', 9.49)


In [80]:
# Назва товару, ціна якого в діапазоні від 3 до 6. Відсортуйте по ціні в порядку зростання

sql_text_7 = """
SELECT prod_name, prod_price
FROM Products
WHERE prod_price BETWEEN 3 AND 6
ORDER BY prod_price 
"""
print('Назва товару, ціна якого в діапазоні від 3 до 6\n')
execute_with_print(sql_text_7)

Назва товару, ціна якого в діапазоні від 3 до 6

('Fish bean bag toy', 3.49)
('Bird bean bag toy', 3.49)
('Rabbit bean bag toy', 3.49)
('Raggedy Ann', 4.99)
('8 inch teddy bear', 5.99)


In [91]:
# Кількість товару, що було продано
sql_text_8 = """
SELECT prod_id, SUM(quantity)
FROM OrderItems
GROUP BY prod_id

UNION

SELECT 'Всього продано: ', SUM(quantity)
FROM OrderItems
"""
print('Кількість товару, що було продано\n')
execute_with_print(sql_text_8)


Кількість товару, що було продано

('BNBG01', 360)
('BNBG02', 360)
('BNBG03', 360)
('BR01', 120)
('BR02', 10)
('BR03', 165)
('RGAN01', 55)
('Всього продано: ', 1430)


In [90]:
# Кількість найменувань товару, ціна якого > 4
sql_text_9 = """
SELECT count(prod_id) FROM Products
WHERE prod_price > 4
"""
print('Кількість найменувань товару, ціна якого > 4\n')
execute_with_print(sql_text_9)

Кількість найменувань товару, ціна якого > 4

(6,)


In [92]:
# 3 найдорожчі товари в базі
sql_text_10 = """
SELECT prod_name, prod_price
FROM Products
ORDER BY prod_price DESC
LIMIT 3
"""
print('Кількість найменувань товару, ціна якого > 4\n')
execute_with_print(sql_text_10_1)

Кількість найменувань товару, ціна якого > 4

('18 inch teddy bear', 11.99)
('King doll', 9.49)
('Queen doll', 9.49)


In [140]:
# Кількість замовлень для кожного клієнта (використовуючи підзапити)
sql_text_11 = """
SELECT cust_id AS c1, SUM(1)
FROM (SELECT cust_id FROM Orders)
GROUP BY cust_id
"""
print('Кількість замовлень для кожного клієнта (використовуючи підзапити)\n')
execute_with_print(sql_text_11)


Кількість замовлень для кожного клієнта (використовуючи підзапити)

('1000000001', 2)
('1000000003', 1)
('1000000004', 1)
('1000000005', 1)


In [141]:
# Список клієнтів та їх замовлень (використовуючи JOIN)
sql_text_12 = """
SELECT cust_name, order_num
FROM Orders
JOIN Customers on Customers.cust_id=Orders.cust_id
"""
print('Список клієнтів та їх замовлень (використовуючи JOIN)\n')
execute_with_print(sql_text_12)

Список клієнтів та їх замовлень (використовуючи JOIN)

('Village Toys', 20005)
('Fun4All', 20006)
('Fun4All', 20007)
('The Toy Store', 20008)
('Village Toys', 20009)


In [146]:
# Список товарів, ціна яких < 5; список товарів, ціна яких >=5; використати UNION для об'єднання
sql_text_13 = """
SELECT prod_name, prod_price
FROM Products
WHERE prod_price < 5

UNION

SELECT prod_name, prod_price 
FROM Products
WHERE prod_price >= 5
"""
print("Список товарів, ціна яких < 5; список товарів, ціна яких >=5; використати UNION для об'єднання\n")
execute_with_print(sql_text_13)

Список товарів, ціна яких < 5; список товарів, ціна яких >=5; використати UNION для об'єднання

('12 inch teddy bear', 8.99)
('18 inch teddy bear', 11.99)
('8 inch teddy bear', 5.99)
('Bird bean bag toy', 3.49)
('Fish bean bag toy', 3.49)
('King doll', 9.49)
('Queen doll', 9.49)
('Rabbit bean bag toy', 3.49)
('Raggedy Ann', 4.99)
